In [1]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

llm = ChatGroq(
    model='llama-3.1-70b-versatile',
    temperature= 0,
    groq_api_key='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'

)
result = llm.invoke('hello')
loader = WebBaseLoader('https://jobs.nike.com/job/R-46907?from=job%20search%20funnel')
page_data=loader.load().pop().page_content
#print(page_data)
prompt_extract= PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTOR:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing thr following keys:
    'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON ( NO PREAMBLE OR MARKDOWN FORMAT):
    """
)
chain_extract= prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
#print(res.content)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

json_parser = JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res


[{'role': 'Lead Software Engineer - 3D Pipeline',
  'experience': '8+ years of experience in 3D pipeline development and tool creation, with a proven track record of technical leadership and team management',
  'skills': ['Python',
   'C++',
   '3D graphics principles',
   'animation',
   'visualization technologies',
   'real-time rendering',
   'Maya',
   '3ds Max',
   'Houdini',
   'Blender',
   'Unreal Engine',
   'Unity'],
  'description': 'We are seeking a highly experienced and visionary Lead 3D Pipeline Engineer to spearhead our 3D pipeline development and innovation within the CP&I team at Nike. In this leadership role, you will not only possess deep technical expertise but also demonstrate exceptional strategic thinking and team management skills.'}]

In [5]:
import pandas as pd
df = pd.read_csv('x.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [20]:
import uuid
import chromadb
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                        metadatas={'links': row['Links']},
                        ids= [str(uuid.uuid4())])


In [23]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas')
links= links[0][:2]
links

[{'links': 'https://example.com/ml-python-portfolio'},
 {'links': 'https://example.com/python-portfolio'}]

In [19]:
job =  json_res[0]
job['skills']

['Python',
 'C++',
 '3D graphics principles',
 'animation',
 'visualization technologies',
 'real-time rendering',
 'Maya',
 '3ds Max',
 'Houdini',
 'Blender',
 'Unreal Engine',
 'Unity']

In [24]:
prompt_email= PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)
chain_email = prompt_email | llm
res =chain_email.invoke({'job_description': job, 'link_list': links})
print(res.content)

Subject: Expert 3D Pipeline Development Solutions for Nike's CP&I Team

Dear Hiring Manager,

I came across the job posting for a Lead Software Engineer - 3D Pipeline at Nike, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your 3D pipeline development goals. With our expertise in creating tailored solutions, we can facilitate the seamless integration of your business processes through automated tools.

Our team at AtliQ has extensive experience in developing innovative solutions using Python, C++, and various visualization technologies. We've worked on numerous projects that involve 3D graphics principles, animation, and real-time rendering. Our expertise in tools like Maya, 3ds Max, Houdini, Blender, Unreal Engine, and Unity can help you streamline your 3D pipeline development.

As a visionary Lead 3D Pipeline Engineer, you're looking for a partner that can help you spearhead innovation within the CP&I team at Nike. AtliQ can be that 